<a href="https://colab.research.google.com/github/Youki/cloud-ai-handson/blob/master/machine_learning/cloud_ai_platform/bigquery_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```
---

# BigQuery ML で出生体重を予測


このノートブックでは、BigQueryの[Publicデータセット](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.natality)を使用して出生時体重を予測しています。

このデータには、1969年から2008年までの米国の[出生に関する詳細](https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details)が含まれています。

BigQueryの詳細は [BigQuery ドキュメント](https://cloud.google.com/bigquery/docs) および [ライブラリリファレンスドキュメント](https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/index.html)を参照してください。

## 事前準備


1.   まだ作成していない場合は [Google Cloud Platform プロジェクト](https://console.cloud.google.com/cloud-resource-manager)を作成します。 
2.   [課金設定](https://support.google.com/cloud/answer/6293499#enable-billing) を有効にします。
3.   [BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery) を有効にします。

In [0]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery

### Google アカウントの認証を実行します。
下記のコードを実行すると、認証コードを取得するための画面へのリンクが表示されるので、そのリンク先へアクセスし、BigQuery への権限を持つアカウントで認証します。その後の画面で表示される認証コードをコピーして Colaboratory のテキストエリアへ入力します。

In [0]:
from google.colab import auth
auth.authenticate_user()
print('認証されました。')

### このノートブックで使用するProject IDを設定します

In [0]:
#@title プロジェクト変数の設定 { run: "auto", display-mode: "form" }
project_id = 'your-project-name' #@param {type:"string"}
client = bigquery.Client(project=project_id)

# BigQuery ML で出生数を予測

このノートブックでは、BigQueryの[Publicデータセット](https://bigquery.cloud.google.com/table/bigquery-public-data:samples.natality)を使用して出生時体重を予測しています。

このデータには、1969年から2008年までの米国の出生に関する詳細が含まれています。
（詳細：https://bigquery.cloud.google.com/table/publicdata:samples.natality?tab=details）

BigQueryの詳細は [BigQuery ドキュメント](https://cloud.google.com/bigquery/docs) および [ライブラリリファレンスドキュメント](https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/index.html)を参照してください。


In [0]:
%%bigquery --project {project_id} data
SELECT *
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 300

In [0]:
import seaborn
from  matplotlib import pyplot as plt
#plt(data)
fg = seaborn.FacetGrid(data=data, hue='plurality', size=6,aspect=1.67)
fg = fg.map(plt.scatter, 'mother_age' ,'weight_pounds').add_legend()
fg = fg.set_axis_labels(x_var="Mother's age", y_var="Weight pounds")

In [0]:
_ = data.hist(column='weight_pounds',by='is_male', layout=(1,2),  sharey=True, sharex=True)

In [0]:
import numpy as np
x = data.gestation_weeks
y = data.weight_pounds
data.plot(kind="scatter",x="gestation_weeks",y="weight_pounds", figsize=[10,6],ylim=0,xlim=20)
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r")
plt.title("Weight pounds by Gestation Weeks.")
plt.show()

## 特徴量の設定

データセットを見てみると、出生時体重を適切に予測するために活用できそうな、いくつかの列があります。BQMLでは、すべての文字列はカテゴリカル機能と見なされ、すべての数値型は連続値と見なされます。
ハッシュ化された年月を追加することで、同様の誕生日の赤ちゃんのデータをトレーニングセットとテストセットにバランスよく分割します。

In [0]:
%%bigquery --project {project_id}
SELECT
    weight_pounds, -- 回帰分析のラベルとして使用します。
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE
  year > 2000
  AND gestation_weeks > 0
  AND mother_age > 0
  AND plurality > 0
  AND weight_pounds > 0
LIMIT 10

## モデルのトレーニング

予測に使用する列を選択することで、BigQueryでモデルを作成（トレーニング）することが可能になります。まず、モデルを保存するためのデータセットが必要になります。 （エラーが発生した場合は、BigQueryコンソールからデータセットを作成してください）。

In [0]:
!gcloud config set project {project_id}
!bq --location=US mk -d demo

デモデータセットの準備が整ったら、線形回帰モデルを作成してモデルを訓練することができます。
実行には約** 4分**かかります。

In [0]:
%%bigquery --project {project_id}
CREATE or REPLACE MODEL demo.babyweight_model_asis
OPTIONS
  (model_type='linear_reg', labels=['weight_pounds']) AS
  
WITH natality_data AS (
  SELECT
    weight_pounds,-- 回帰分析のラベル
    is_male AS is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
  FROM
    publicdata.samples.natality
  WHERE
    year > 2000
    AND gestation_weeks > 0
    AND mother_age > 0
    AND plurality > 0
    AND weight_pounds > 0
)

SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    natality_data
WHERE
  MOD(hashmonth, 4) < 3  -- select 75% of the data as training
LIMIT 19

In [0]:
data.weight_pounds.mad() # MAE, MAD = Mean Absolute Error, Mean Absolute Difference

## モデルの精度

上記の平均絶対誤差 (`mean_absolute_error`) 値 `1.05`. よりも下回れば、期待した精度になっていると言えます。平均絶対誤差は、予測ラベルと実際のラベルとの差の平均値です。


In [0]:
%%bigquery --project {project_id} df
select * from ml.EVALUATE(MODEL demo.babyweight_model_asis);

## BQML モデルで予測を実行

訓練されたモデルで値を予測することが可能になりました。

`ml.predict`関数を利用すると、モデルの出力予測列名は` predicted_ <label_column_name> `になります。

In [0]:
%%bigquery --project {project_id}
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (SELECT
        weight_pounds,
        is_male,
        mother_age,
        plurality AS plurality,
        gestation_weeks
      FROM
        publicdata.samples.natality
      WHERE
        year > 2000
        AND gestation_weeks > 0
        AND mother_age > 0
        AND plurality > 0
        AND weight_pounds > 0
    ))
LIMIT 10

28際の母親から38週で生まれた赤ちゃんの体重を以下のように予測してみます。

In [0]:

%%bigquery --project {project_id}
SELECT
  *
FROM
  ml.PREDICT(MODEL demo.babyweight_model_asis,
      (SELECT
          TRUE AS is_male,
          28 AS mother_age,
          1 AS plurality,
          38 AS gestation_weeks
    ))